# DinnerMatch
*** By Alexander Munoz ***

In [103]:
import pandas as pd
import numpy as np
from itertools import izip_longest

In [100]:
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # Taken from itertools recipes:
    # https://docs.python.org/2/library/itertools.html#recipes
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx
    args = [iter(iterable)] * n
    return izip_longest(fillvalue=fillvalue, *args)

In [81]:
data = pd.read_csv('data.csv')
data = data.ix[:,[1,2,6,8,9]]
data.columns = ['name', 'email', 'times', 'friend_name', 'friend_email']
data = data.set_index('name')
print data.shape
data.head()

(73, 4)


,email,times,friend_name,friend_email
name,,,,
Eesha Khare,ekhare@college.harvard.edu,"Wednesday, 3/8 6:00pm, Wednesday, 3/8 6:30pm...",Anne Deng,annedeng@college.harvard.edu
Kamara Swaby,kswaby01@college.harvard.edu,"Monday, 2/27 5:00pm, Monday, 2/27 6:00pm, Mo...",NaN,NaN
Jacob Meyerson,jacobmeyerson@college.harvard.edu,"Friday, 3/3 6:30pm, Tuesday, 3/7 6:30pm, Wed...",NaN,NaN
Pieter weemaes,Pweemaes@college.harvard.edu,"Monday, 2/27 5:00pm, Monday, 2/27 6:00pm, Mo...",NaN,NaN
Russell Pekala,russellpekala@college.harvard.edu,"Monday, 2/27 6:00pm, Monday, 2/27 6:30pm, Tu...",NaN,NaN


In [206]:
times_dict = dict() #key is name, value is list of available times
for i in range(len(data['times'])):
    time_list_buffer = data['times'][i].split('pm, ') #split times
    time_list_buffer[-1] = time_list_buffer[-1][:-2] #remove last 'pm'
    times_dict[data.index.values[i]] = time_list_buffer

In [207]:
times_options = set()
for person in times_dict:
    times_options = times_options.union(set(times_dict[person]))
times_options = list(times_options)
np.random.shuffle(times_options)
print times_options

['Sunday, 3/5  6:30', 'Wednesday, 3/1  6:00', 'Wednesday, 3/8  6:30', 'Sunday, 3/5  6:00', 'Monday, 3/6  5:00', 'Thursday, 3/9  6:00', 'Tuesday, 3/7  5:00', 'Saturday, 3/4  6:00', 'Monday, 3/6  6:00', 'Monday, 2/27  6:00', 'Saturday, 3/4  5:00', 'Tuesday, 3/7  6:00', 'Tuesday, 2/28  5:00', 'Tuesday, 2/28  6:00', 'Thursday, 3/9  5:00', 'Sunday, 3/5  5:00', 'Tuesday, 3/7  6:30', 'Thursday, 3/9  6:30', 'Wednesday, 3/8  6:00', 'Thursday, 3/2  5:00', 'Monday, 2/27  5:00', 'Wednesday, 3/8  5:00', 'Wednesday, 3/1  6:30', 'Monday, 3/6  6:30', 'Friday, 3/3  6:00', 'Wednesday, 3/1  5:00', 'Thursday, 3/2  6:30', 'Saturday, 3/4  6:30', 'Tuesday, 2/28  6:30', 'Friday, 3/3  5:00', 'Monday, 2/27  6:30', 'Thursday, 3/2  6:00', 'Friday, 3/3  6:30']


In [208]:
result_dict = dict()
for curr_time in times_options:
    curr_time_availables = []
    for person in times_dict:
        if curr_time in times_dict[person]:
            curr_time_availables.append(person)
    if len(curr_time_availables) == 1:
        pass
    else:
        np.random.shuffle(curr_time_availables)
        if len(curr_time_availables) % 3 == 0 or len(curr_time_availables) % 3 == 2:
            result_dict[curr_time] = list(grouper(curr_time_availables, 3))
        else:
            result_dict[curr_time] = list(grouper(curr_time_availables, 4))
        for p in curr_time_availables:
            times_dict.pop(p) #remove people who have been picked, so they don't get picked twice

In [209]:
times_to_pop = []
for t in result_dict:
    if result_dict[t] == []:
        times_to_pop.append(t)
for p in times_to_pop:
    result_dict.pop(p)
for t in result_dict:
    print t
    print result_dict[t]
    print

Saturday, 3/4  5:00
[('Gita Abhiraman', 'Elaine', None)]

Thursday, 3/9  6:00
[('Cecil ', 'Anna Lea Albright', None)]

Saturday, 3/4  6:00
[('Sarah Wu', 'Celeste Mendoza', 'Uju Nwosu'), ('Noemi Urquiza', 'Jullian Duran', None)]

Monday, 3/6  5:00
[('Cecilia Yao', 'Pieter weemaes', 'Lethu Ntshinga'), ('Arlesia McGowan', 'Andrew Mayo', None)]

Sunday, 3/5  6:30
[('Eric Po', 'Chinaza', 'Francesca Noelette '), ('Katie Smilow ', 'Brad Riew', 'Brooke Bourgeois'), ('Andrea Schoenberg', 'Kamara Swaby', 'Siqi Liu'), ('Justin Fox', 'Ying Guo', 'Joey Kubicki'), ('Serena', 'Walker Maeder-York', 'Amanda Brandt'), ('Stephanie Guo', 'Steph Ferrarie', 'Jason Goodman'), ('Ben Chen', 'Devon Black', 'Christian Floyd'), ('Cass Hastie', 'Lucas Cassels', 'Russell Pekala'), ('Eva DiIanni-Miller', 'Jenna Gray', 'Nakoa Farrant'), ('Jen Xu', 'Peter Tian', 'Victoria'), ("Camille N'Diaye-Muller", 'Chris Jury', 'Michael Ge')]

Wednesday, 3/8  6:30
[('noemi valdez', 'Eesha Khare', 'Molly Leavens'), ('William Chang'

In [210]:
#ensure no one leftover
if not bool(times_dict):
    print "No one left over!"
else:
    print "SOME PEOPLE UNGROUPED"
    print times_dict

No one left over!
